### Visualisierung und Auswertung der Gleichzeitigkeit
#### Nils Bleich

In [2]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import seaborn as sns
from main import read_json_obj
from datetime import datetime
from StationData import get_station
%matplotlib notebook

Einlesen des aktuell auszuwertenden Datesatzes aus MYSQL. Definiert welcher Datensatz wird in der JSON_Config

In [3]:
setup_dict = read_json_obj("C:/Users/nilsb/PycharmProjects/Simultanity/JSON_Config.JSON")
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
                       .format(host=setup_dict["database"]["hostname"],db=setup_dict["database"]["dbname"],
                        user=setup_dict["database"]["uname"],
                        pw=setup_dict["database"]["pwd"]))
db = '{Standort}_historical'.format(Standort=setup_dict['simulation']['station'])
df = pd.read_sql(db, engine)
df.set_index(['datetime_index'], inplace=True)

ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'luedenscheid_historical' at line 1")
[SQL: luedenscheid_historical]
(Background on this error at: https://sqlalche.me/e/14/f405)

Funktion zum Ploten einer Heatmap, welche die Gleichzeitigkeit darstellt. Input data muss in der Form bereits Zeitlich angepasst sein.

In [3]:
def draw_heatmap(data,name, save=False):
    fig = plt.figure(figsize=[10,5], layout='tight')
    normalized_data=(data-data.min())/(data.max()-data.min())
    newdf = normalized_data[['power_total','PV_output','WEA_Power']].copy()
    newdf['date'] = data.index.date
    newdf['time'] = data.index.time
    newdf.drop(columns=['PV_output','WEA_Power'], inplace=True)
    newdf = newdf.pivot('time', 'date', 'power_total')
    #ax = sns.heatmap(newdf, cmap='turbo', linewidths=.005, linecolor='black')
    ax = sns.heatmap(newdf, cmap='turbo',linewidths=.1, linecolor='black', xticklabels=16, yticklabels=12)
    ax.set_title(name)
    if save:
        fig.savefig(fname='{name}.svg'.format(name=name))

In [4]:
yeardf = df['2018-01-01':'2018-12-31']
draw_heatmap(yeardf,name='Gleichzeitigkeit',save=True)

<IPython.core.display.Javascript object>

Schreiben aller KPI Datenbanken in Excel Dateien

In [6]:
#standort_list = ['st. peter ording', 'marnitz', 'leck', 'greifswald', 'schwerin','angermuende','doernick','bremervoerde' ]
standort_list = ['augsburg','werl','waibstadt','luedenscheid']
for standort in standort_list:
    db = '{Standort}_historical_kpi'.format(Standort=standort)
    kpi_df = pd.read_sql(db, engine)
    kpi_df.to_excel('C:/Users/nilsb/Data/Wetterdaten/Auswertung/{Standort}_kpi.xlsx'.format(Standort=standort), sheet_name='sheet0', index=True)


Funktion zum Erstellen der KPI_matrix

In [6]:
def kpi_matrix(data, station ):
    years = (201001010000, 201101010000, 201201010000, 201301010000, 201401010000, 201501010000, 201601010000,
                 201701010000, 201801010000, 201901010000, 201912312350)
    kpi_dict = {2010: {}, 2011: {}, 2012: {}, 2013: {}, 2014: {}, 2015: {}, 2016: {}, 2017: {}, 2018: {}, 2019: {}}
    station_dict = get_station(station=station)
    for i in range(0, 10):
        dtstart = datetime.strptime(str(years[i]), '%Y%m%d%H%M')
        dtend = datetime.strptime(str(years[i + 1]), '%Y%m%d%H%M')
        yeardf = data[dtstart:dtend]
        kpi_dict[dtstart.year]['Volllaststunden PV'] = yeardf['PV_output'].sum() / 6 / (
                    station_dict['pv_cap'] / 0.83333)
        kpi_dict[dtstart.year]['Volllaststunden Wind'] = yeardf['WEA_Power'].sum() / 6 / station_dict[
            'pv_cap']  # PV_cap = WEA Nennleistung. Ist hier gleichwertig
        if setup_dict['simulation']["include_raw_data"]:
            kpi_dict[dtstart.year]['Mittlere Windgeschwindigkeit auf Nabenhöhe'] = yeardf['wind_speed_hubheight'].mean()
            kpi_dict[dtstart.year]['Summe der Solarenstrahlung'] = (yeardf['direct'].sum() + yeardf[
                'diffuse'].sum()) / 6 / 1000
        kpi_dict[dtstart.year]['g > 0.5 in %'] = sum((yeardf['power_total'] / station_dict['pv_cap']) > 1) / yeardf.shape[
            0] * 100
        over = yeardf['power_total'] > station_dict['pv_cap']
        e_cap = yeardf['power_total'].copy()
        e_cap[over] = station_dict['pv_cap']
        e_over = sum(yeardf['power_total'][over])
        e_total = yeardf['power_total'].sum()
        kpi_dict[dtstart.year]['Verlust bei g > 0.5 in %'] = (e_total - e_cap.sum()) / e_total * 100
        kpi_dict[dtstart.year]['Verlust bei g > 0.5 normiert auf Nennleistung'] = (e_total - e_cap.sum())/6/station_dict['pv_cap']
        kpi_dict[dtstart.year]['Missing Datapoints'] = yeardf['power_total'].isnull().sum()
        kpi_dict[dtstart.year]['Missing Datapoints in %'] = yeardf['power_total'].isnull().sum() / yeardf.shape[0] *100
        # db = '{Standort}_historical'.format(Standort=setup_dict['simulation']['station'])
    kpi_db = pd.DataFrame.from_dict(kpi_dict, orient='index')
    return kpi_db

Erneute Berechnung aller KPI Matrizen. Viel SQL Laden & Schreiben -> Sollte überlegt ausgeführt werden

In [24]:
standort_list = ['st. peter ording', 'marnitz', 'leck', 'greifswald', 'schwerin','angermuende','doernick','bremervoerde' ]
for standort in standort_list:
    db = '{Standort}_historical'.format(Standort=standort)
    df = pd.read_sql(db, engine)
    df.set_index(['datetime_index'], inplace=True)
    kpi_df = kpi_matrix(data = df, station=standort)
    kpi_df.to_sql('{Standort}_kpi'.format(Standort=standort), engine, index=True, if_exists='replace')

In [17]:
#Monatsplot
years = (201001010000, 201101010000, 201201010000, 201301010000, 201401010000, 201501010000, 201601010000,
                 201701010000, 201801010000, 201901010000, 201912312350)
fig = plt.figure(figsize=[6,4], layout='tight')
ax = fig.add_subplot(1, 1, 1)
sns.set_theme(color_codes=True)
pv_array = np.empty([10,12])
wind_array = np.empty([10,12])
total_array = np.empty([10,12])
for i in range(0, 10):
    dtstart = datetime.strptime(str(years[i]), '%Y%m%d%H%M')
    dtend = datetime.strptime(str(years[i+1]), '%Y%m%d%H%M')
    yeardf = df[dtstart:dtend]
    newdf = yeardf[['power_total','PV_output','WEA_Power']].copy()
    newdf['month'] = newdf.index.month
    #monthly_dict={}
    for month in range(1,13):
        temp = newdf.loc[newdf['month']==month].sum()
        pv_array[i][month-1]= temp['PV_output']
        wind_array[i][month-1]= temp['WEA_Power']
        total_array[i][month-1]= temp['power_total']
pv_mean = np.empty([12,3])
wind_mean = np.empty([12,3])
total_mean = np.empty([12,3])
pv_array=pv_array/pv_array.max()
wind_array=wind_array/wind_array.max()
total_array=total_array/total_array.max()
for month in range(1,13):
    pv_mean[month-1,0] = pv_array[:,month-1].mean()
    pv_mean[month-1,1] = pv_array[:,month-1].min()
    pv_mean[month-1,2] = pv_array[:,month-1].max()

    wind_mean[month-1,0] = wind_array[:,month-1].mean()
    wind_mean[month-1,1] = wind_array[:,month-1].min()
    wind_mean[month-1,2] = wind_array[:,month-1].max()

    total_mean[month-1,0] = total_array[:,month-1].mean()
    total_mean[month-1,1] = total_array[:,month-1].min()
    total_mean[month-1,2] = total_array[:,month-1].max()

    #wind_array[i][month-1]
    #total_array[i][month-1]
x = np.linspace(1,12,12)
ax.plot(x, pv_mean[:,0], linewidth=2, label='mittlere PV Erzeugung')
ax.fill_between(x, pv_mean[:,1].flatten(), pv_mean[:,2].flatten(),alpha=.5, linewidth=0, label='Min. & Max.')
ax.plot(x, wind_mean[:,0], linewidth=2, label='mittlere Windkraft Erzeugung')
ax.fill_between(x, wind_mean[:,1].flatten(), wind_mean[:,2].flatten(),alpha=.5, linewidth=0, label='Min. & Max.')
ax.set_title('Monatssummen der Erneuerbaren Erzeugung')
ax.set_xlabel('Monat des Jahres')
ax.set_ylabel('Monatliche Einspeisung normiert')
ax.legend(loc='upper right',prop={'size': 8})
fig.savefig(fname='{name}_monat.svg'.format(name=setup_dict['simulation']['station']))

<IPython.core.display.Javascript object>

In [10]:
df.shape

(525887, 7)

In [11]:
#Wochenplot
years = (201001010000, 201101010000, 201201010000, 201301010000, 201401010000, 201501010000, 201601010000,201701010000, 201801010000, 201901010000, 201912312350)
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
sns.set_theme(color_codes=True)
pv_array = np.empty([10,53])
wind_array = np.empty([10,53])
total_array = np.empty([10,53])
for i in range(0, 10):
    dtstart = datetime.strptime(str(years[i]), '%Y%m%d%H%M')
    dtend = datetime.strptime(str(years[i+1]), '%Y%m%d%H%M')
    yeardf = df[dtstart:dtend]
    newdf = yeardf[['power_total','PV_output','WEA_Power']].copy()
    newdf['week'] = newdf.index.isocalendar().week
    #monthly_dict={}
    for week in range(1,53):
        temp = newdf.loc[newdf['week']==week].sum()
        pv_array[i][week-1]= temp['PV_output']
        wind_array[i][week-1]= temp['WEA_Power']
        total_array[i][week-1]= temp['power_total']
pv_mean = np.empty([53,3])
wind_mean = np.empty([53,3])
total_mean = np.empty([53,3])
pv_array=pv_array/pv_array.max()
wind_array=wind_array/wind_array.max()
total_array=total_array/total_array.max()
for week in range(1,54):
    pv_mean[week-1,0] = pv_array[:,week-1].mean()
    pv_mean[week-1,1] = pv_array[:,week-1].min()
    pv_mean[week-1,2] = pv_array[:,week-1].max()

    wind_mean[week-1,0] = wind_array[:,week-1].mean()
    wind_mean[week-1,1] = wind_array[:,week-1].min()
    wind_mean[week-1,2] = wind_array[:,week-1].max()

    total_mean[week-1,0] = total_array[:,week-1].mean()
    total_mean[week-1,1] = total_array[:,week-1].min()
    total_mean[week-1,2] = total_array[:,week-1].max()

    #wind_array[i][month-1]
    #total_array[i][month-1]
x = np.linspace(1,53,53)
ax.plot(x[:52], pv_mean[:52,0], linewidth=2, label='mittlere PV Erzeugung')
ax.fill_between(x, pv_mean[:,1].flatten(), pv_mean[:,2].flatten(),alpha=.5, linewidth=0,label='Min. und Max.')
ax.plot(x[:52], wind_mean[:52,0], linewidth=2, label='mittlere Windkraft Erzeugung')
ax.fill_between(x, wind_mean[:,1].flatten(), wind_mean[:,2].flatten(),alpha=.5, linewidth=0,label='Min. und Max.')
ax.set_title('Wochensummen der Erneuerbaren Erzeugung')
ax.set_xlabel('Woche des des Jahres')
ax.set_ylabel('Wochen Einspeisung normiert')
ax.legend(loc='best')
fig.savefig(fname='{name}_woche.svg'.format(name=setup_dict['simulation']['station']))

<IPython.core.display.Javascript object>

In [8]:
#Tagesplot
years = (201001010000, 201101010000, 201201010000, 201301010000, 201401010000, 201501010000, 201601010000,
                 201701010000, 201801010000, 201901010000, 201912312350)
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
sns.set_theme(color_codes=True)
pv_array = np.zeros([10,365])
wind_array = np.zeros([10,365])
total_array = np.zeros([10,365])
over_array = np.zeros([10,365])
for i in range(0, 10):
    dtstart = datetime.strptime(str(years[i]), '%Y%m%d%H%M')
    dtend = datetime.strptime(str(years[i+1]), '%Y%m%d%H%M')
    yeardf = df[dtstart:dtend]
    newdf = yeardf[['power_total','PV_output','WEA_Power']].copy()
    newdf['day'] = newdf.index.dayofyear
    #monthly_dict={}
    for day in range(1,366):
        temp = newdf.loc[newdf['day']==day]
        pv_array[i][day-1]= temp['PV_output'].sum()
        wind_array[i][day-1]= temp['WEA_Power'].sum()
        total_array[i][day-1]= temp['power_total'].sum()
        over_array[i][day-1]=(temp['power_total']/newdf['power_total'].max()>0.5).sum() /(24*6) *100
pv_mean = np.zeros([365,3])
wind_mean = np.zeros([365,3])
total_mean = np.zeros([365,3])
over_sum = np.zeros([365,3])
pv_array=pv_array/pv_array.max()
wind_array=wind_array/wind_array.max()
total_array=total_array/total_array.max()
for day in range(1,366):
    pv_mean[day-1,0] = pv_array[:,day-1].mean()
    pv_mean[day-1,1] = pv_array[:,day-1].min()
    pv_mean[day-1,2] = pv_array[:,day-1].max()

    wind_mean[day-1,0] = wind_array[:,day-1].mean()
    wind_mean[day-1,1] = wind_array[:,day-1].min()
    wind_mean[day-1,2] = wind_array[:,day-1].max()

    total_mean[day-1,0] = total_array[:,day-1].mean()
    total_mean[day-1,1] = total_array[:,day-1].min()
    total_mean[day-1,2] = total_array[:,day-1].max()

    over_sum[day-1,0] = over_array[:,day-1].mean()
    over_sum[day-1,1] = over_array[:,day-1].min()
    over_sum[day-1,2] = over_array[:,day-1].max()
x = np.linspace(1,365,365)
ax.plot(x[:365], pv_mean[:365,0], linewidth=2, label='mittlere PV Erzeugung')
ax.fill_between(x, pv_mean[:365,1].flatten(), pv_mean[:365,2].flatten(),alpha=.5, linewidth=0,label='Min. und Max.')
ax.plot(x[:365], wind_mean[:365,0], linewidth=2, label='mittlere Windkraft Erzeugung')
ax.fill_between(x, wind_mean[:365,1].flatten(), wind_mean[:365,2].flatten(),alpha=.5, linewidth=0,label='Min. und Max.')
ax.set_title('Tagessummen der Erneuerbaren Erzeugung')
ax.set_xlabel('Tage  des Jahres')
ax.set_ylabel('Tages Einspeisung normiert')
ax.legend(loc='best', fancybox=True,prop={'size': 6})
#fig.savefig(fname='{name}_tag.svg'.format(name=setup_dict['simulation']['station']))


<IPython.core.display.Javascript object>

In [9]:
fig2 = plt.figure()
ax2 = fig2.add_subplot(1, 1, 1)
ax2.plot(x[:365], over_sum[:365,0], linewidth=2, label='mean g>0,5 in %')
ax2.fill_between(x, over_sum[:365,1].flatten(), over_sum[:365,2].flatten(),alpha=.5, linewidth=0,label='Min. und Max.')
ax2.set_title('Mittelwert der Zeiten in denen Gleichezeitigkeit > 0.5')
ax2.set_xlabel('Tage des Jahres')
ax2.set_ylabel('Teil des Tages an dem g>0,5 in %')
ax2.legend(loc='best')
#fig2.savefig(fname='{name}_Gleichzeitigkeit_tag.svg'.format(name=setup_dict['simulation']['station']))

<IPython.core.display.Javascript object>

In [14]:
over_sum

array([[ 9.72222222,  0.        , 27.08333333],
       [ 5.48611111,  0.        , 22.91666667],
       [ 7.63888889,  0.        , 20.13888889],
       ...,
       [ 6.31944444,  0.        , 18.05555556],
       [10.13888889,  0.        , 22.91666667],
       [ 5.76388889,  0.        , 22.91666667]])

In [ ]:
standort_list = ['st. peter ording', 'marnitz', 'leck', 'greifswald', 'schwerin','angermuende','doernick','bremervoerde' ]
fig = plt.figure(figsize=[10,20])
for i,standort in enumerate(standort_list):
    ax = fig.add_subplot(8, 1, i+1)
    db = '{Standort}_historical'.format(Standort=standort)
    df = pd.read_sql(db, engine)
    df.set_index(['datetime_index'], inplace=True)
    df['g'] = df['power_total'] / df['power_total'].max()
    sns.histplot(df['g'],label=standort,legend=True, bins=70)
    ax.legend(loc='best')
fig.savefig(fname='{name}_hist.svg'.format(name=setup_dict['simulation']['station']))

In [12]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
day_array = np.zeros([365,10])
day_pv_array = np.zeros([365,10])
day_wind_array = np.zeros([365,10])
years = (201001010000, 201101010000, 201201010000, 201301010000, 201401010000, 201501010000, 201601010000,
                 201701010000, 201801010000, 201901010000, 201912312350)
for i in range(0, 10):
    dtstart = datetime.strptime(str(years[i]), '%Y%m%d%H%M')
    dtend = datetime.strptime(str(years[i + 1]), '%Y%m%d%H%M')
    yeardf = df[dtstart:dtend].copy()
    normalized_data= yeardf / yeardf.max()
    newdf = normalized_data[['power_total','PV_output','WEA_Power']].copy()
    newdf['day'] = yeardf.index.dayofyear
    newdf['over'] = newdf['power_total'] > 0.5
    for day in range(1,365):
        temp =newdf.loc[newdf['day']==day]
        day_pv_array[day-1][i] = temp['PV_output'].sum()/6
        day_wind_array[day-1][i] = temp['WEA_Power'].sum()/6
        day_array[day-1][i] = temp['over'].sum()/(6*24) *100
    x = np.linspace(1,364,364)
plt.step(x,day_array[:364,:], linewidth=0.5)
ax.set_title('Gleichzeitigkeit')
ax.set_xlabel('Tage des Jahres')
ax.set_ylabel('Teil des Tages an dem g>0,5 in %')
#fig.savefig(fname='{}_tag.svg'.format(name=setup_dict['simulation']['station']))

<IPython.core.display.Javascript object>

Text(0, 0.5, 'Teil des Tages an dem g>0,5 in %')

In [10]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
week_array = np.zeros([52,10])
years = (201001010000, 201101010000, 201201010000, 201301010000, 201401010000, 201501010000, 201601010000,
                 201701010000, 201801010000, 201901010000, 201912312350)
for i in range(0, 10):
    dtstart = datetime.strptime(str(years[i]), '%Y%m%d%H%M')
    dtend = datetime.strptime(str(years[i + 1]), '%Y%m%d%H%M')
    yeardf = df[dtstart:dtend].copy()
    normalized_data= yeardf / yeardf.max()
    newdf = normalized_data[['power_total','PV_output','WEA_Power']].copy()
    newdf['week'] = yeardf.index.isocalendar().week
    newdf['over'] = newdf['power_total'] > 0.5
    for week in range(1,53):
        temp =newdf.loc[newdf['week']==week]
        week_array[week-1][i] = temp['over'].sum()/(6*24*7) *100
    x = np.linspace(1,52,52)
plt.step(x,week_array[:52,:], linewidth=0.5)
ax.set_title('Gleichzeitigkeit')
ax.set_xlabel('Wochen des Jahres')
ax.set_ylabel('Anteil der Woche an dem g>0,5 in %')


<IPython.core.display.Javascript object>

Text(0, 0.5, 'Anteil der Woche an dem g>0,5 in %')

In [13]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.grid()
ax.plot(day_array.mean(axis=1)[:364],label='g>0,5 in %')
ax2=ax.twinx()
ax2.plot(day_pv_array.mean(axis=1)[:364], color='tab:orange',label='PV')
ax2.plot(day_wind_array.mean(axis=1)[:364], color='tab:cyan',label='Wind')
ax2.set_ylabel('Volllaststunden an betrachtetem Tag')
ax.set_title('Mittelwert der Zeiten in denen Gleichezeitigkeit > 0.5')
ax.set_xlabel('Tage des Jahres')
ax.set_ylabel('Teil des Tages an dem g>0,5 in %')
ax.legend()
ax2.legend()
fig.savefig(fname='{name}_Gleichzeitigkeit_Erzeugung.svg'.format(name=setup_dict['simulation']['station']))

<IPython.core.display.Javascript object>

In [14]:
plt.close('all')

C:\Users\nilsb\AppData\Local\Temp\ipykernel_18548\2445104545.py:1: MatplotlibDeprecationWarning: The close_event function was deprecated in Matplotlib 3.6 and will be removed two minor releases later. Use callbacks.process('close_event', CloseEvent(...)) instead.
  plt.close('all')


In [7]:
def get_g(data, g):
    over = data['power_total'] > (data['power_total'].max()*g)
    return over.sum()/data.shape[0] * 100

In [9]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.grid()
for g in range(50,101):
    g_result = get_g(df,g/100)
    ax.scatter(g/100,g_result,color='cornflowerblue',label='g')
ax.set_title('Testrun')
ax.set_xlabel('g')
ax.set_ylabel('Zeitpunkte in denen P>g*P %')

<IPython.core.display.Javascript object>

Text(0, 0.5, 'Zeitpunkte in denen P>g*P %')

In [6]:
def get_ratio(data, r):
    ans_dict = {}
    new_df = data.copy()
    new_df['PV_output'] = new_df['PV_output'] * r
    new_df['power_total'] = new_df['PV_output']+ new_df['WEA_Power']
    over = new_df['power_total'] > new_df['WEA_Power'].max()
    ans_dict['g'] = over.sum()/data.shape[0] * 100
    e_cap = new_df['power_total'].copy()
    e_cap[over] = new_df['WEA_Power'].max()
    e_total = new_df['power_total'].sum()
    ans_dict['loss in %'] =(e_total - e_cap.sum()) / e_total * 100
    ans_dict['loss normiert'] =(e_total - e_cap.sum())/6/new_df['WEA_Power'].max()/10
    #print(f'Etotal: {e_total}')
    #print(f'ECap_sum {e_cap.sum()}')
    #print(new_df['WEA_Power'].max())
    return ans_dict

In [15]:
db = '{Standort}_historical'.format(Standort='angermuende')
df_sued = pd.read_sql(db, engine)
df_sued.set_index(['datetime_index'], inplace=True)
db = '{Standort}_e_w_historical'.format(Standort='angermuende')
df_ew = pd.read_sql(db, engine)
df_ew.set_index(['datetime_index'], inplace=True)

In [9]:
fig = plt.figure(figsize=[12,5])
ax = fig.add_subplot(1, 3, 1)
ax1 = fig.add_subplot(1, 3, 2)
ax2 = fig.add_subplot(1, 3, 3)
sns.set_theme(color_codes=True)
for r in range(1,200):
    r_result = get_ratio(df,r/100)
    #print(r)
    #print(r_result['loss normiert'])
    #print('________________')
    ax.scatter(r/100,r_result['g'],color='tab:orange',label='g',s=2)
    ax1.scatter(r/100,r_result['loss in %'],color='tab:blue',label='g',s=2)
    ax2.scatter(r/100,r_result['loss normiert'],color='tab:green',label='g',s=2)
ax1.set_title('Einfluss der installierten PV Leitung, bei konstanter Windpark und Umspannwerk Leistung')
ax.set_xlabel('Verhältnis von PV zu Wind Leistung')
ax.set_ylabel('Zeitpunkte in denen P>0,5 %')
ax1.set_xlabel('Verhältnis von PV zu Wind Leistung')
ax1.set_ylabel('Verlust durch Abregelung in %')
ax2.set_xlabel('Verhältnis von PV zu Wind Leistung')
ax2.set_ylabel('Verlust durch Abregelung normiert')
fig.savefig(fname='{name}_Ratio.svg'.format(name='Angermünde_Süd'))

<IPython.core.display.Javascript object>

In [18]:
plt.close('all')

C:\Users\nilsb\AppData\Local\Temp\ipykernel_20864\2445104545.py:1: MatplotlibDeprecationWarning: The close_event function was deprecated in Matplotlib 3.6 and will be removed two minor releases later. Use callbacks.process('close_event', CloseEvent(...)) instead.
  plt.close('all')


In [20]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
df_diff = df_sued['PV_output']-df_ew['PV_output']
df_diff['2011-01-01':'2012-01-01'].plot()

<IPython.core.display.Javascript object>

<AxesSubplot: xlabel='datetime_index'>

In [14]:
df_ew

'angermuende_e_w_historical'